In [2]:
%pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 29.2 MB/s  0:00:00 eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

# Path to your census codebook
file_path = "/Users/kwatchomahinanda/Downloads/census_codebook.csv"

try:
    census_df = pd.read_csv(file_path)
    
    print("\n--- CENSUS CODEBOOK SUMMARY ---")
    print(f"Total Variables: {census_df.shape[0]}")
    print(f"Columns: {census_df.columns.tolist()}")
    
    print("\n--- FIRST 3 ROWS ---")
    print(census_df.head(3))
    
    print("\n--- DETAILED LOOK AT FIRST VARIABLE ---")
    for col in census_df.columns:
        print(f"{col}: {census_df.iloc[0][col]}")

except Exception as e:
    print(f"Error: {e}")


--- CENSUS CODEBOOK SUMMARY ---
Total Variables: 496956
Columns: ['title', 'description', 'vintage', 'variable', 'label']

--- FIRST 3 ROWS ---
                                               title  \
0  Apr 1989 Current Population Survey: Basic Monthly   
1  Apr 1989 Current Population Survey: Basic Monthly   
2  Apr 1989 Current Population Survey: Basic Monthly   

                                         description  vintage  variable  \
0  To provide estimates of employment, unemployme...     1989  A_ABSREA   
1  To provide estimates of employment, unemployme...     1989   A_AG_NA   
2  To provide estimates of employment, unemployme...     1989     A_AGE   

                                         label  
0  Labor Force-reasons for absence, pay status  
1     Indus.&Occ.-agriculture, non-agriculture  
2                              Demographic-age  

--- DETAILED LOOK AT FIRST VARIABLE ---
title: Apr 1989 Current Population Survey: Basic Monthly
description: To provide estimates o

In [6]:
import pandas as pd
import os

# Paths to your BWDC files
edu_path = "/Users/kwatchomahinanda/Downloads/explore_data_edu_01/edu_01.csv"
dict_path = "/Users/kwatchomahinanda/Downloads/explore_data_edu_01/data-dictionary.xlsx"

print("\n--- BWDC EDUCATION DATA ---")
try:
    edu_df = pd.read_csv(edu_path)
    print(f"Shape: {edu_df.shape}")
    print(f"Columns: {edu_df.columns.tolist()[:10]}... (Total: {len(edu_df.columns)})")
    print("\nFirst 2 rows:")
    print(edu_df.head(2))
except Exception as e:
    print(f"Error loading CSV: {e}")

print("\n--- DATA DICTIONARY (EXCEL) ---")
try:
    dict_df = pd.read_excel(dict_path)
    print(f"Columns: {dict_df.columns.tolist()}")
    print("\nFirst 3 entries:")
    print(dict_df.head(3))
except Exception as e:
    print(f"Error loading Excel: {e}")


--- BWDC EDUCATION DATA ---
Shape: (312, 5)
Columns: ['sex', 'educational_attainment', 'year', 'race_or_ethnicity', 'rate']... (Total: 5)

First 2 rows:
      sex       educational_attainment  year race_or_ethnicity  rate
0    Male  Bachelor's or higher degree  2014             Black  21.0
1  Female  Bachelor's or higher degree  2017          Hispanic  18.6

--- DATA DICTIONARY (EXCEL) ---
Columns: ['column_name', 'visual name', 'variable name on visual', 'visual_unique_id', 'data table', 'publisher', 'description']

First 3 entries:
         column_name                                        visual name  \
0               year  Median Total Assets for Households, by Race/Et...   
1               race  Median Total Assets for Households, by Race/Et...   
2  education_buckets  Median Total Assets for Households, by Race/Et...   

      variable name on visual visual_unique_id                     data table  \
0                        Year           adt_01  \nSummary Extract Public Data

In [8]:
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions

# 1. Setup ChromaDB (Local Persistence)
client = chromadb.PersistentClient(path="./invest_atlanta_vault")

# Using a standard embedding model (will download automatically on first run)
emb_fn = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

# Create a collection for Education Stats
edu_collection = client.get_or_create_collection(name="bwdc_education", embedding_function=emb_fn)

# 2. Load the CSV
df = pd.read_csv("/Users/kwatchomahinanda/Downloads/explore_data_edu_01/edu_01.csv")

documents = []
metadatas = []
ids = []

print("Vectorizing data... please wait.")

for idx, row in df.iterrows():
    # Construct a human-readable sentence for the AI to "read"
    sentence = (f"In {row['year']}, the educational attainment rate for {row['race_or_ethnicity']} "
                f"{row['sex']} individuals labeled as '{row['educational_attainment']}' was {row['rate']}%.")
    
    documents.append(sentence)
    metadatas.append({"year": int(row['year']), "race": row['race_or_ethnicity']})
    ids.append(f"edu_row_{idx}")

# 3. Add to ChromaDB
edu_collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

print(f"Successfully vectorized {len(documents)} rows into the local vault.")

# Force-save the data to the disk (specific to Jupyter environments)
client.heartbeat() 
print("Vault is locked and saved.")

Vectorizing data... please wait.
Successfully vectorized 312 rows into the local vault.
Vault is locked and saved.


In [9]:
# 1. Define your question
query = "What is the bachelor's degree rate for Black men?"

# 2. Search the vault
# n_results=3 means "give me the top 3 closest matches"
results = edu_collection.query(
    query_texts=[query],
    n_results=3
)

# 3. Print the results
print(f"--- RESULTS FOR: '{query}' ---")
for i, doc in enumerate(results['documents'][0]):
    print(f"Match {i+1}: {doc}")

--- RESULTS FOR: 'What is the bachelor's degree rate for Black men?' ---
Match 1: In 2017, the educational attainment rate for Black Male individuals labeled as 'Bachelor's or higher degree' was 22.6%.
Match 2: In 2018, the educational attainment rate for Black Male individuals labeled as 'Bachelor's or higher degree' was 23.7%.
Match 3: In 2019, the educational attainment rate for Black Male individuals labeled as 'Bachelor's or higher degree' was 24.4%.


In [10]:
import os
print(f"Your database is currently hidden here: {os.getcwd()}/invest_atlanta_vault")

Your database is currently hidden here: /Users/kwatchomahinanda/DATASCI 531/datasci531spring2026/Lecture_04/invest_atlanta_vault


In [12]:
# The '!' tells Jupyter to run this in the system terminal
!pip install ollama

In [ ]:
import ollama

def ask_invest_atlanta(question):
    # 1. Retrieve the top 3 relevant facts from your local ChromaDB vault
    results = edu_collection.query(
        query_texts=[question],
        n_results=3
    )
    
    # 2. Combine the retrieved facts into a single "Context" block
    context = "\n".join(results['documents'][0])
    
    # 3. Create the prompt for Llama 3
    # We tell the AI to ONLY use the data we just found.
    prompt = f"""
    You are a professional Data Assistant for Invest Atlanta. 
    Use the following specific statistics to answer the user's question. 
    If the data isn't in the context, say you don't have that specific data point.

    DATA CONTEXT:
    {context}

    USER QUESTION: 
    {question}

    YOUR RESPONSE:
    """

    # 4. Send it to your local Llama 3 model
    response = ollama.generate(model='llama3-grants', prompt=prompt)
    
    return response['response']

# --- TEST IT OUT ---
user_q = "Can you summarize the bachelor's degree trends for Black men between 2017 and 2019?"
print(f"Assistant is thinking...\n")
print(ask_invest_atlanta(user_q))

Assistant is thinking...



In [1]:
import pandas as pd
import os

# ============================================
# REPLACE THIS with your actual file path
# ============================================
file_path = "/Users/kwatchomahinanda/Downloads/census_codebook.csv"

print("STEP 1: Loading Census Codebook")
print("="*70)

# Check if file exists
if not os.path.exists(file_path):
    print(f"❌ ERROR: File not found at {file_path}")
    print("\nPlease check:")
    print("1. File name is correct")
    print("2. File is in the right location")
    print("3. You have the full path")
else:
    print(f"✅ File found!")
    
    # Check file size
    file_size = os.path.getsize(file_path) / (1024 * 1024)  # Convert to MB
    print(f"📦 File size: {file_size:.2f} MB")
    
    print("\n" + "="*70)
    print("ATTEMPTING TO LOAD FIRST 10 ROWS...")
    print("="*70)
    
    try:
        # Try reading first 10 rows
        if file_path.endswith('.csv'):
            df_sample = pd.read_csv(file_path, nrows=10)
        elif file_path.endswith('.xlsx'):
            df_sample = pd.read_excel(file_path, nrows=10)
        else:
            print("❌ Unknown file format. Please use .csv or .xlsx")
            df_sample = None
        
        if df_sample is not None:
            print("✅ Successfully loaded sample!")
            print(f"\n📊 Dataset Info:")
            print(f"   - Columns: {len(df_sample.columns)}")
            print(f"   - Sample rows: {len(df_sample)}")
            
            print(f"\n📋 COLUMN NAMES:")
            for i, col in enumerate(df_sample.columns, 1):
                print(f"   {i}. {col}")
            
            print(f"\n🔍 FIRST 5 ROWS:")
            print(df_sample.head().to_string())
            
            print(f"\n📈 DATA TYPES:")
            print(df_sample.dtypes.to_string())
            
            print(f"\n✨ SAMPLE DATA FROM EACH COLUMN:")
            for col in df_sample.columns:
                sample_value = df_sample[col].iloc[0] if len(df_sample) > 0 else "N/A"
                print(f"   {col}: '{sample_value}'")
            
    except Exception as e:
        print(f"❌ Error loading file: {type(e).__name__}")
        print(f"   Details: {str(e)}")
        print("\n" + "="*70)
        print("TRYING ALTERNATIVE METHOD (CHUNKED READING)...")
        print("="*70)
        
        try:
            # Alternative: Read in chunks
            if file_path.endswith('.csv'):
                chunk_iterator = pd.read_csv(file_path, chunksize=100)
                first_chunk = next(chunk_iterator)
                
                print("✅ Loaded first chunk (100 rows)!")
                print(f"\n📊 Chunk Info:")
                print(f"   - Shape: {first_chunk.shape}")
                print(f"   - Columns: {len(first_chunk.columns)}")
                
                print(f"\n📋 COLUMN NAMES:")
                for i, col in enumerate(first_chunk.columns, 1):
                    print(f"   {i}. {col}")
                
                print(f"\n🔍 FIRST 5 ROWS:")
                print(first_chunk.head().to_string())
                
            else:
                print("❌ Cannot chunk-read Excel files easily.")
                print("   Suggestion: Convert to CSV first or use smaller sample")
                
        except Exception as e2:
            print(f"❌ Chunked reading also failed: {type(e2).__name__}")
            print(f"   Details: {str(e2)}")
            print("\n💡 SUGGESTIONS:")
            print("   1. Make sure the file isn't corrupted")
            print("   2. Try opening it in Excel/Numbers to verify it works")
            print("   3. Convert to CSV if it's Excel")
            print("   4. Check if you have enough RAM")

print("\n" + "="*70)
print("END OF STEP 1")
print("="*70)
print("\n📤 Please copy and paste ALL of the output above!")

STEP 1: Loading Census Codebook
✅ File found!
📦 File size: 359.63 MB

ATTEMPTING TO LOAD FIRST 10 ROWS...
✅ Successfully loaded sample!

📊 Dataset Info:
   - Columns: 5
   - Sample rows: 10

📋 COLUMN NAMES:
   1. title
   2. description
   3. vintage
   4. variable
   5. label

🔍 FIRST 5 ROWS:
                                               title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [2]:
import pandas as pd

file_path = "/Users/kwatchomahinanda/Downloads/census_codebook.csv"

print("STEP 2: Sample & Pattern Analysis")
print("="*70)

# Load a larger sample (1000 rows)
print("Loading first 1,000 rows...")
df_sample = pd.read_csv(file_path, nrows=1000)

print(f"✅ Loaded {len(df_sample)} rows\n")

# Basic stats
print("📊 DATASET STATISTICS:")
print(f"   Total rows in sample: {len(df_sample)}")
print(f"   Unique variables: {df_sample['variable'].nunique()}")
print(f"   Unique titles: {df_sample['title'].nunique()}")
print(f"   Year range: {df_sample['vintage'].min()} - {df_sample['vintage'].max()}")

# Show unique survey titles
print(f"\n📋 SURVEY TYPES IN SAMPLE:")
print(f"   Found {df_sample['title'].nunique()} different surveys:")
for i, title in enumerate(df_sample['title'].unique()[:10], 1):
    count = len(df_sample[df_sample['title'] == title])
    print(f"   {i}. {title[:70]}... ({count} variables)")

# Look at variable naming patterns
print(f"\n🔤 VARIABLE NAME PATTERNS:")
print(f"   Sample variable codes:")
for var in df_sample['variable'].head(20):
    print(f"   - {var}")

# Look at label patterns
print(f"\n🏷️ LABEL PATTERNS (first 30):")
for i, label in enumerate(df_sample['label'].head(30), 1):
    print(f"   {i}. {label}")

# Check for keywords in labels
print(f"\n🔍 KEYWORD ANALYSIS:")
keywords = {
    'income': 0,
    'population': 0,
    'housing': 0,
    'employment': 0,
    'education': 0,
    'age': 0,
    'race': 0,
    'poverty': 0,
    'household': 0,
    'median': 0
}

for label in df_sample['label']:
    if pd.notna(label):  # Check if not null
        label_lower = str(label).lower()
        for keyword in keywords:
            if keyword in label_lower:
                keywords[keyword] += 1

print("   Keyword occurrences in sample:")
for keyword, count in sorted(keywords.items(), key=lambda x: x[1], reverse=True):
    print(f"   - '{keyword}': {count} times")

# Show some examples of each keyword
print(f"\n📌 EXAMPLE LABELS BY KEYWORD:")
example_keywords = ['income', 'housing', 'population', 'employment', 'education']

for keyword in example_keywords:
    print(f"\n   Examples with '{keyword}':")
    examples = df_sample[df_sample['label'].str.contains(keyword, case=False, na=False)]['label'].head(5)
    for ex in examples:
        print(f"   - {ex}")

print("\n" + "="*70)
print("END OF STEP 2")
print("="*70)
print("\n📤 Please paste this output!")

STEP 2: Sample & Pattern Analysis
Loading first 1,000 rows...
✅ Loaded 1000 rows

📊 DATASET STATISTICS:
   Total rows in sample: 1000
   Unique variables: 238
   Unique titles: 5
   Year range: 1989 - 1989

📋 SURVEY TYPES IN SAMPLE:
   Found 5 different surveys:
   1. Apr 1989 Current Population Survey: Basic Monthly... (238 variables)
   2. Aug 1989 Current Population Survey: Basic Monthly... (238 variables)
   3. Dec 1989 Current Population Survey: Basic Monthly... (238 variables)
   4. Feb 1989 Current Population Survey: Basic Monthly... (238 variables)
   5. Jan 1989 Current Population Survey: Basic Monthly... (48 variables)

🔤 VARIABLE NAME PATTERNS:
   Sample variable codes:
   - A_ABSREA
   - A_AG_NA
   - A_AGE
   - A_ANYWK
   - A_AVAIL
   - A_CHKWJ
   - A_CIVLF
   - A_CLSWKR
   - A_DSCWK
   - A_DTCLWK
   - A_DTIND
   - A_DTOCC
   - A_EARNRT
   - A_EMP
   - A_EMPHRS
   - A_ENRCHK
   - A_ENRLW
   - A_ERNEL
   - A_ERNLWT
   - A_EXPLF

🏷️ LABEL PATTERNS (first 30):
   1. Labor Forc

In [3]:
import pandas as pd

file_path = "/Users/kwatchomahinanda/Downloads/census_codebook.csv"

print("STEP 3: Exploring Full Dataset Variety")
print("="*70)

# Get total row count
print("Counting total rows (this may take a moment)...")
total_rows = sum(1 for _ in open(file_path)) - 1  # -1 for header
print(f"✅ Total rows in file: {total_rows:,}")

# Sample from different parts of the file
print(f"\n📊 Sampling from different sections...")

# Beginning (first 500)
df_beginning = pd.read_csv(file_path, nrows=500)

# Middle (skip to middle, read 500)
skip_to_middle = total_rows // 2
df_middle = pd.read_csv(file_path, skiprows=range(1, skip_to_middle), nrows=500)

# End (read last 500)
skip_to_end = total_rows - 500
df_end = pd.read_csv(file_path, skiprows=range(1, skip_to_end), nrows=500)

# Combine samples
df_combined = pd.concat([df_beginning, df_middle, df_end], ignore_index=True)

print(f"✅ Sampled {len(df_combined)} rows from beginning, middle, and end")

# Analyze all survey types
print(f"\n📋 ALL SURVEY TYPES IN FILE:")
unique_titles = df_combined['title'].unique()
print(f"   Found {len(unique_titles)} different survey types:\n")
for i, title in enumerate(unique_titles[:20], 1):  # Show first 20
    count = len(df_combined[df_combined['title'] == title])
    print(f"   {i}. {title[:80]}... ({count} vars)")

if len(unique_titles) > 20:
    print(f"\n   ... and {len(unique_titles) - 20} more survey types")

# Year range
print(f"\n📅 YEAR RANGE:")
print(f"   Oldest: {df_combined['vintage'].min()}")
print(f"   Newest: {df_combined['vintage'].max()}")

# Expanded keyword analysis on diverse sample
print(f"\n🔍 KEYWORD ANALYSIS (DIVERSE SAMPLE):")
keywords = {
    'income': 0, 'earnings': 0, 'wage': 0, 'salary': 0,
    'population': 0, 'demographic': 0,
    'housing': 0, 'house': 0, 'rent': 0, 'mortgage': 0,
    'employment': 0, 'labor': 0, 'work': 0, 'occupation': 0,
    'education': 0, 'school': 0, 'enrollment': 0,
    'age': 0, 'sex': 0, 'race': 0, 'ethnicity': 0, 'hispanic': 0,
    'poverty': 0, 'poor': 0,
    'household': 0, 'family': 0,
    'health': 0, 'insurance': 0, 'disability': 0,
    'veteran': 0, 'military': 0,
    'median': 0, 'total': 0, 'percent': 0
}

for label in df_combined['label']:
    if pd.notna(label):
        label_lower = str(label).lower()
        for keyword in keywords:
            if keyword in label_lower:
                keywords[keyword] += 1

print("   Top keywords:")
sorted_keywords = sorted(keywords.items(), key=lambda x: x[1], reverse=True)
for keyword, count in sorted_keywords[:25]:  # Top 25
    if count > 0:
        print(f"   - '{keyword}': {count} times")

# Show diverse label examples
print(f"\n📌 DIVERSE LABEL EXAMPLES:")
print(f"   Random sample of 30 labels from across the file:\n")
sample_labels = df_combined['label'].dropna().sample(min(30, len(df_combined)), random_state=42)
for i, label in enumerate(sample_labels, 1):
    print(f"   {i}. {label}")

print("\n" + "="*70)
print("END OF STEP 3")
print("="*70)

STEP 3: Exploring Full Dataset Variety
Counting total rows (this may take a moment)...
✅ Total rows in file: 584,888

📊 Sampling from different sections...
✅ Sampled 1000 rows from beginning, middle, and end

📋 ALL SURVEY TYPES IN FILE:
   Found 4 different survey types:

   1. Apr 1989 Current Population Survey: Basic Monthly... (238 vars)
   2. Aug 1989 Current Population Survey: Basic Monthly... (238 vars)
   3. Dec 1989 Current Population Survey: Basic Monthly... (24 vars)
   4. Wave 7 Survey of Income and Program Participation - 2008 Panel: Topical Modules... (500 vars)

📅 YEAR RANGE:
   Oldest: 1989
   Newest: 2008

🔍 KEYWORD ANALYSIS (DIVERSE SAMPLE):
   Top keywords:
   - 'labor': 195 times
   - 'house': 158 times
   - 'household': 158 times
   - 'work': 96 times
   - 'age': 71 times
   - 'demographic': 69 times
   - 'family': 67 times
   - 'total': 56 times
   - 'earnings': 48 times
   - 'income': 46 times
   - 'school': 26 times
   - 'rent': 24 times
   - 'enrollment': 19 tim

In [ ]:
CATEGORY_SCHEMA = {
    "01_DEMOGRAPHICS": {
        "description": "Age, sex, race, ethnicity, family composition",
        "keywords": ["age", "sex", "race", "ethnicity", "hispanic", "demographic", 
                     "family", "household", "relationship", "marital"],
        "subcategories": [
            "Age & Gender",
            "Race & Ethnicity", 
            "Family & Household Composition",
            "Marital Status"
        ]
    },
    
    "02_INCOME_EARNINGS": {
        "description": "Wages, salary, earnings, total income",
        "keywords": ["income", "earnings", "wage", "salary", "pay", "revenue"],
        "subcategories": [
            "Household Income",
            "Personal Earnings",
            "Wage & Salary",
            "Family Income"
        ]
    },
    
    "03_EMPLOYMENT_LABOR": {
        "description": "Employment status, occupation, industry, work hours",
        "keywords": ["employment", "labor", "work", "occupation", "industry", 
                     "job", "employed", "unemployed", "worker", "class of worker"],
        "subcategories": [
            "Employment Status",
            "Occupation",
            "Industry",
            "Work Hours & Conditions"
        ]
    },
    
    "04_HOUSING": {
        "description": "Housing units, rent, mortgage, property value",
        "keywords": ["housing", "house", "rent", "mortgage", "property", 
                     "dwelling", "residence", "apartment", "owner", "renter"],
        "subcategories": [
            "Housing Units",
            "Rent & Rental Property",
            "Mortgage & Home Ownership",
            "Property Value"
        ]
    },
    
    "05_EDUCATION": {
        "description": "School enrollment, educational attainment, degrees",
        "keywords": ["education", "school", "enrollment", "degree", "college", 
                     "university", "grade", "attainment", "diploma"],
        "subcategories": [
            "School Enrollment",
            "Educational Attainment",
            "Degrees & Credentials"
        ]
    },
    
    "06_POVERTY_ASSISTANCE": {
        "description": "Poverty status, government assistance programs",
        "keywords": ["poverty", "poor", "assistance", "welfare", "tanf", 
                     "snap", "food stamp", "medicaid", "ssi", "general assist"],
        "subcategories": [
            "Poverty Status",
            "Government Assistance Programs",
            "Social Security & Benefits"
        ]
    },
    
    "07_HEALTH_DISABILITY": {
        "description": "Health insurance, disability, medical coverage",
        "keywords": ["health", "insurance", "disability", "medical", "medicare", 
                     "medicaid", "coverage", "doctor", "dentist"],
        "subcategories": [
            "Health Insurance",
            "Disability Status",
            "Medical Access & Care"
        ]
    },
    
    "08_ASSETS_WEALTH": {
        "description": "Savings, stocks, investments, assets, debt",
        "keywords": ["asset", "saving", "stock", "investment", "account", 
                     "fund", "bond", "debt", "owe", "equity"],
        "subcategories": [
            "Savings & Accounts",
            "Stocks & Investments",
            "Assets & Property",
            "Debt"
        ]
    },
    
    "09_VETERAN_MILITARY": {
        "description": "Military service, veteran status, military benefits",
        "keywords": ["veteran", "military", "service", "armed forces", 
                     "va", "retirement pay"],
        "subcategories": [
            "Veteran Status",
            "Military Service",
            "Military Benefits"
        ]
    },
    
    "10_GEOGRAPHY": {
        "description": "Location codes, state, county, tract identifiers",
        "keywords": ["geography", "fips", "state", "county", "tract", 
                     "location", "msa", "cbsa"],
        "subcategories": [
            "State & County",
            "Census Tract",
            "Metropolitan Area"
        ]
    },
    
    "11_WEIGHTS_TECHNICAL": {
        "description": "Statistical weights, flags, technical variables",
        "keywords": ["weight", "flag", "allocation", "imputation", "recode", 
                     "check item", "interviewer"],
        "subcategories": [
            "Survey Weights",
            "Allocation Flags",
            "Technical Variables"
        ]
    },
    
    "12_OTHER": {
        "description": "Variables that don't fit other categories",
        "keywords": [],
        "subcategories": []
    }
}

In [5]:
import pandas as pd

file_path = "/Users/kwatchomahinanda/Downloads/census_codebook.csv"

print("STEP 5.5: Improved Categorization")
print("="*70)

# UPDATED category schema with fixes
CATEGORY_SCHEMA = {
    "01_DEMOGRAPHICS": {
        "keywords": ["age", "sex", "race", "ethnicity", "hispanic", "demographic", 
                     "family", "household", "relationship", "marital", "person"],
        "exclude": ["household income", "family income"],
        "priority": 5  # Medium priority
    },
    "02_INCOME_EARNINGS": {
        "keywords": ["income", "earnings", "wage", "salary", "pay", "revenue"],
        "exclude": [],
        "priority": 7  # High priority - specific terms
    },
    "03_EMPLOYMENT_LABOR": {
        "keywords": ["employment", "labor", "work", "occupation", "industry", 
                     "job", "employed", "unemployed", "worker", "class of worker",
                     "hours worked", "full-time", "part-time", "indus", "occ",
                     "agriculture", "non-agriculture"],  # ADDED: indus, occ, agriculture
        "exclude": [],
        "priority": 6  # High priority
    },
    "04_HOUSING": {
        "keywords": ["housing", "house", "rent", "mortgage", "property", 
                     "dwelling", "residence", "apartment", "owner", "renter",
                     "real estate", "home"],
        "exclude": ["household"],
        "priority": 7  # High priority - specific terms
    },
    "05_EDUCATION": {
        "keywords": ["education", "school", "enrollment", "degree", "college", 
                     "university", "grade", "attainment", "diploma", "student"],
        "exclude": [],
        "priority": 7  # High priority - specific terms
    },
    "06_POVERTY_ASSISTANCE": {
        "keywords": ["poverty", "poor", "assistance", "welfare", "tanf", 
                     "snap", "food stamp", "medicaid", "ssi", "general assist",
                     "public assistance", "government program"],
        "exclude": [],
        "priority": 8  # Highest priority - very specific
    },
    "07_HEALTH_DISABILITY": {
        "keywords": ["health", "insurance", "disability", "medical", "medicare", 
                     "coverage", "doctor", "dentist", "hospital", "disabled"],
        "exclude": [],
        "priority": 7  # High priority
    },
    "08_ASSETS_WEALTH": {
        "keywords": ["asset", "saving", "stock", "investment", "account", 
                     "fund", "bond", "debt", "owe", "equity", "wealth",
                     "bank", "financial", "dividend", "interest", "margin"],
        "exclude": [],
        "priority": 6  # Medium-high priority
    },
    "09_VETERAN_MILITARY": {
        "keywords": ["veteran", "military", "service", "armed forces", 
                     "va", "retirement pay", "army", "navy"],
        "exclude": [],
        "priority": 7  # High priority - specific
    },
    "10_GEOGRAPHY": {
        "keywords": ["geography", "fips", "state", "county", "tract", 
                     "location", "msa", "cbsa", "region", "place", "cmsa",
                     "metropolitan", "central city"],  # More specific geo terms
        "exclude": ["allocation"],  # ADDED: Don't catch allocation flags
        "priority": 6  # Medium-high priority
    },
    "11_WEIGHTS_TECHNICAL": {
        "keywords": ["weight", "flag", "allocation", "imputation", "recode", 
                     "check item", "interviewer", "topcode", "edit", "final weight"],
        "exclude": [],
        "priority": 4  # Lower priority - catch-all for technical stuff
    }
}

def categorize_variable_improved(label, variable_code):
    """
    Improved categorization with priority system
    """
    if pd.isna(label):
        return "12_OTHER", 0.0, "No label"
    
    label_lower = str(label).lower()
    
    # Check each category
    category_matches = {}
    
    for category, info in CATEGORY_SCHEMA.items():
        score = 0
        matched_keywords = []
        
        # Check for excluded keywords first
        excluded = False
        for exclude_word in info["exclude"]:
            if exclude_word in label_lower:
                excluded = True
                break
        
        if excluded:
            continue
        
        # Count matching keywords
        for keyword in info["keywords"]:
            if keyword in label_lower:
                score += 1
                matched_keywords.append(keyword)
        
        if score > 0:
            # Weighted score = matches * priority
            weighted_score = score * info["priority"]
            category_matches[category] = {
                "score": score,
                "weighted_score": weighted_score,
                "keywords": matched_keywords
            }
    
    # Return category with highest weighted score
    if category_matches:
        best_category = max(category_matches, key=lambda x: category_matches[x]["weighted_score"])
        best_info = category_matches[best_category]
        return best_category, best_info["score"], ", ".join(best_info["keywords"])
    else:
        return "12_OTHER", 0.0, "No matches"

# Test on sample
print("Loading sample (2000 rows)...")
df_sample = pd.read_csv(file_path, nrows=2000)

print(f"✅ Loaded {len(df_sample)} rows\n")

# Categorize with improved function
print("Categorizing variables with improved logic...")
results = df_sample.apply(
    lambda row: categorize_variable_improved(row['label'], row['variable']), 
    axis=1
)

df_sample['category'] = results.apply(lambda x: x[0])
df_sample['category_confidence'] = results.apply(lambda x: x[1])
df_sample['matched_keywords'] = results.apply(lambda x: x[2])

print("✅ Categorization complete!\n")

# Show distribution
print("📊 CATEGORY DISTRIBUTION:")
category_counts = df_sample['category'].value_counts()
for category, count in category_counts.items():
    percentage = (count / len(df_sample)) * 100
    print(f"   {category}: {count} variables ({percentage:.1f}%)")

# Check our problem cases
print(f"\n🔍 CHECKING PREVIOUSLY MISCLASSIFIED VARIABLES:")
print("="*70)

problem_vars = ['AXUNCOV', 'A_AG_NA', 'A_IOELIG']
for var in problem_vars:
    if var in df_sample['variable'].values:
        row = df_sample[df_sample['variable'] == var].iloc[0]
        print(f"\nVariable: {row['variable']}")
        print(f"Label: {row['label']}")
        print(f"Category: {row['category']}")
        print(f"Matched: {row['matched_keywords']}")

# Show 12_OTHER category
print(f"\n\n📌 VARIABLES IN '12_OTHER' CATEGORY:")
print("="*70)
other_vars = df_sample[df_sample['category'] == '12_OTHER'][['variable', 'label']].head(15)
if len(other_vars) > 0:
    for idx, row in other_vars.iterrows():
        print(f"\nVariable: {row['variable']}")
        print(f"Label: {row['label']}")
else:
    print("✅ No variables in OTHER category!")

print("\n" + "="*70)
print("END OF STEP 5.5")
print("="*70)
print("\n✅ Fixed issues:")
print("   1. Added 'indus', 'occ', 'agriculture' to EMPLOYMENT_LABOR")
print("   2. Excluded 'allocation' from GEOGRAPHY")
print("   3. Added priority system to handle conflicts")
print("\n📤 Please review - are the problem variables fixed?")

STEP 5.5: Improved Categorization
Loading sample (2000 rows)...
✅ Loaded 2000 rows

Categorizing variables with improved logic...
✅ Categorization complete!

📊 CATEGORY DISTRIBUTION:
   03_EMPLOYMENT_LABOR: 771 variables (38.6%)
   01_DEMOGRAPHICS: 476 variables (23.8%)
   11_WEIGHTS_TECHNICAL: 339 variables (17.0%)
   02_INCOME_EARNINGS: 158 variables (7.9%)
   10_GEOGRAPHY: 104 variables (5.2%)
   05_EDUCATION: 102 variables (5.1%)
   09_VETERAN_MILITARY: 33 variables (1.7%)
   04_HOUSING: 9 variables (0.4%)
   07_HEALTH_DISABILITY: 8 variables (0.4%)

🔍 CHECKING PREVIOUSLY MISCLASSIFIED VARIABLES:

Variable: AXUNCOV
Label: Earnings-union/employee assoc. contract (allocation flag)
Category: 11_WEIGHTS_TECHNICAL
Matched: flag, allocation

Variable: A_AG_NA
Label: Indus.&Occ.-agriculture, non-agriculture
Category: 03_EMPLOYMENT_LABOR
Matched: indus, occ, agriculture, non-agriculture

Variable: A_IOELIG
Label: Indus.&Occ.-I&O eligibility
Category: 03_EMPLOYMENT_LABOR
Matched: indus, occ

In [7]:
import pandas as pd
import time

file_path = "/Users/kwatchomahinanda/Downloads/census_codebook.csv"

print("STEP 6: Processing Full Dataset (584,888 rows)")
print("="*70)
print("This will take 10-20 minutes - be patient!")
print("="*70)

# Same improved category schema
CATEGORY_SCHEMA = {
    "01_DEMOGRAPHICS": {
        "keywords": ["age", "sex", "race", "ethnicity", "hispanic", "demographic", 
                     "family", "household", "relationship", "marital", "person"],
        "exclude": ["household income", "family income"],
        "priority": 5
    },
    "02_INCOME_EARNINGS": {
        "keywords": ["income", "earnings", "wage", "salary", "pay", "revenue"],
        "exclude": [],
        "priority": 7
    },
    "03_EMPLOYMENT_LABOR": {
        "keywords": ["employment", "labor", "work", "occupation", "industry", 
                     "job", "employed", "unemployed", "worker", "class of worker",
                     "hours worked", "full-time", "part-time", "indus", "occ",
                     "agriculture", "non-agriculture"],
        "exclude": [],
        "priority": 6
    },
    "04_HOUSING": {
        "keywords": ["housing", "house", "rent", "mortgage", "property", 
                     "dwelling", "residence", "apartment", "owner", "renter",
                     "real estate", "home"],
        "exclude": ["household"],
        "priority": 7
    },
    "05_EDUCATION": {
        "keywords": ["education", "school", "enrollment", "degree", "college", 
                     "university", "grade", "attainment", "diploma", "student"],
        "exclude": [],
        "priority": 7
    },
    "06_POVERTY_ASSISTANCE": {
        "keywords": ["poverty", "poor", "assistance", "welfare", "tanf", 
                     "snap", "food stamp", "medicaid", "ssi", "general assist",
                     "public assistance", "government program"],
        "exclude": [],
        "priority": 8
    },
    "07_HEALTH_DISABILITY": {
        "keywords": ["health", "insurance", "disability", "medical", "medicare", 
                     "coverage", "doctor", "dentist", "hospital", "disabled"],
        "exclude": [],
        "priority": 7
    },
    "08_ASSETS_WEALTH": {
        "keywords": ["asset", "saving", "stock", "investment", "account", 
                     "fund", "bond", "debt", "owe", "equity", "wealth",
                     "bank", "financial", "dividend", "interest", "margin"],
        "exclude": [],
        "priority": 6
    },
    "09_VETERAN_MILITARY": {
        "keywords": ["veteran", "military", "service", "armed forces", 
                     "va", "retirement pay", "army", "navy"],
        "exclude": [],
        "priority": 7
    },
    "10_GEOGRAPHY": {
        "keywords": ["geography", "fips", "state", "county", "tract", 
                     "location", "msa", "cbsa", "region", "place", "cmsa",
                     "metropolitan", "central city"],
        "exclude": ["allocation"],
        "priority": 6
    },
    "11_WEIGHTS_TECHNICAL": {
        "keywords": ["weight", "flag", "allocation", "imputation", "recode", 
                     "check item", "interviewer", "topcode", "edit", "final weight"],
        "exclude": [],
        "priority": 4
    }
}

def categorize_variable_improved(label, variable_code):
    """Categorization function with priority system"""
    if pd.isna(label):
        return "12_OTHER", 0.0, "No label"
    
    label_lower = str(label).lower()
    category_matches = {}
    
    for category, info in CATEGORY_SCHEMA.items():
        score = 0
        matched_keywords = []
        
        excluded = False
        for exclude_word in info["exclude"]:
            if exclude_word in label_lower:
                excluded = True
                break
        
        if excluded:
            continue
        
        for keyword in info["keywords"]:
            if keyword in label_lower:
                score += 1
                matched_keywords.append(keyword)
        
        if score > 0:
            weighted_score = score * info["priority"]
            category_matches[category] = {
                "score": score,
                "weighted_score": weighted_score,
                "keywords": matched_keywords
            }
    
    if category_matches:
        best_category = max(category_matches, key=lambda x: category_matches[x]["weighted_score"])
        best_info = category_matches[best_category]
        return best_category, best_info["score"], ", ".join(best_info["keywords"])
    else:
        return "12_OTHER", 0.0, "No matches"

# Process in chunks to track progress
chunk_size = 50000
output_file = "census_codebook_categorized.csv"

print(f"\nReading file in chunks of {chunk_size:,} rows...")
print(f"Will save results to: {output_file}\n")

start_time = time.time()
first_chunk = True

for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunk_size)):
    chunk_start = time.time()
    
    # Categorize
    results = chunk.apply(
        lambda row: categorize_variable_improved(row['label'], row['variable']), 
        axis=1
    )
    
    chunk['category'] = results.apply(lambda x: x[0])
    chunk['category_confidence'] = results.apply(lambda x: x[1])
    chunk['matched_keywords'] = results.apply(lambda x: x[2])
    
    # Save to CSV
    if first_chunk:
        chunk.to_csv(output_file, index=False, mode='w')
        first_chunk = False
    else:
        chunk.to_csv(output_file, index=False, mode='a', header=False)
    
    chunk_time = time.time() - chunk_start
    rows_processed = (i + 1) * chunk_size
    elapsed = time.time() - start_time
    
    print(f"Chunk {i+1}: Processed {len(chunk):,} rows in {chunk_time:.1f}s")
    print(f"   Total so far: {rows_processed:,} rows | Elapsed: {elapsed/60:.1f} min")
    print()

total_time = time.time() - start_time

print("="*70)
print("PROCESSING COMPLETE")
print("="*70)
print(f"Total time: {total_time/60:.1f} minutes")
print(f"Saved to: {output_file}")

# Load and analyze final results
print(f"\nLoading final results for analysis...")
df_final = pd.read_csv(output_file)

print(f"\nFINAL CATEGORY DISTRIBUTION:")
category_counts = df_final['category'].value_counts()
for category, count in category_counts.items():
    percentage = (count / len(df_final)) * 100
    print(f"   {category}: {count:,} variables ({percentage:.1f}%)")

print(f"\nTotal variables categorized: {len(df_final):,}")
print(f"File saved: {output_file}")

print("\n" + "="*70)
print("END OF STEP 6")
print("="*70)

STEP 6: Processing Full Dataset (584,888 rows)
This will take 10-20 minutes - be patient!

Reading file in chunks of 50,000 rows...
Will save results to: census_codebook_categorized.csv

Chunk 1: Processed 50,000 rows in 1.0s
   Total so far: 50,000 rows | Elapsed: 0.0 min

Chunk 2: Processed 50,000 rows in 1.1s
   Total so far: 100,000 rows | Elapsed: 0.0 min

Chunk 3: Processed 50,000 rows in 1.0s
   Total so far: 150,000 rows | Elapsed: 0.1 min

Chunk 4: Processed 50,000 rows in 1.0s
   Total so far: 200,000 rows | Elapsed: 0.1 min

Chunk 5: Processed 50,000 rows in 1.0s
   Total so far: 250,000 rows | Elapsed: 0.1 min

Chunk 6: Processed 50,000 rows in 1.0s
   Total so far: 300,000 rows | Elapsed: 0.1 min

Chunk 7: Processed 50,000 rows in 1.4s
   Total so far: 350,000 rows | Elapsed: 0.1 min

Chunk 8: Processed 50,000 rows in 1.0s
   Total so far: 400,000 rows | Elapsed: 0.2 min

Chunk 9: Processed 50,000 rows in 1.1s
   Total so far: 450,000 rows | Elapsed: 0.2 min

Chunk 10: Pro

In [8]:
import pandas as pd

categorized_file = "census_codebook_categorized.csv"

print("STEP 7: Investigating OTHER Category")
print("="*70)

# Load the categorized data
df = pd.read_csv(categorized_file)

# Filter to OTHER category
df_other = df[df['category'] == '12_OTHER']

print(f"Total in OTHER category: {len(df_other):,}")
print(f"Percentage of total: {(len(df_other)/len(df))*100:.1f}%")

# Sample 50 random examples
print(f"\nRandom sample of 50 variables from OTHER category:")
print("="*70)

sample = df_other.sample(min(50, len(df_other)), random_state=42)
for i, row in sample.iterrows():
    print(f"\nVariable: {row['variable']}")
    print(f"Label: {row['label']}")
    print(f"Survey: {row['title'][:60]}...")

# Analyze patterns in OTHER
print("\n\nPattern Analysis:")
print("="*70)

# Check for common words
from collections import Counter
all_labels = ' '.join(df_other['label'].dropna().astype(str).str.lower()).split()
word_counts = Counter(all_labels)

print("\nTop 30 most common words in OTHER category labels:")
for word, count in word_counts.most_common(30):
    print(f"   {word}: {count}")

# Check survey titles
print("\n\nSurvey distribution in OTHER category:")
survey_counts = df_other['title'].value_counts().head(10)
for survey, count in survey_counts.items():
    print(f"   {survey[:70]}... : {count}")

print("\n" + "="*70)
print("END OF STEP 7")
print("="*70)

STEP 7: Investigating OTHER Category
Total in OTHER category: 63,030
Percentage of total: 12.7%

Random sample of 50 variables from OTHER category:

Variable: TM9385
Label: Are the Names of Any Emp.s Listed for th
Survey: Wave 8 Survey of Income and Program Participation - 1992 Pan...

Variable: IO10003
Label: Imp. Own Int. 100-103
Survey: Wave 6 Survey of Income and Program Participation - 1992 Pan...

Variable: ISE22322
Label: Imp. Proprietorship/Partnership
Survey: Wave 8 Survey of Income and Program Participation - 1990 Pan...

Variable: T39AMT
Label: GI: Amount of pension/retirement lump sums (ISS Code 39)
Survey: Wave 1 Survey of Income and Program Participation - 2004 Pan...

Variable: TM7566
Label: TM7566
Survey: Wave 9 Survey of Income and Program Participation - 1992 Pan...

Variable: EASST01
Label: ED: Federal Pell Grant
Survey: Wave 12 Survey of Income and Program Participation - 1996 Pa...

Variable: RWS1
Label: GI: Reason for stopping WIC the first time
Survey: Wave 8 Sur

In [ ]:
"""
Census Codebook Categorization Script
--------------------------------------
This script categorizes Census variables into 12 thematic categories
for easier retrieval in the Invest Atlanta RAG system.

Input: census_codebook.csv (raw Census variable codebook)
Output: census_codebook_categorized.csv (codebook with category assignments)

Author: Kwatcho Mahinanda
Date: January 2026
"""

import pandas as pd
import time

# CONFIGURATION

INPUT_FILE = "/Users/kwatchomahinanda/Downloads/census_codebook.csv"
OUTPUT_FILE = "census_codebook_categorized.csv"
CHUNK_SIZE = 50000  # Process in chunks to manage memory

# CATEGORY SCHEMA

# Each category has:
# - keywords: terms that indicate this category
# - exclude: terms that disqualify a match
# - priority: higher priority wins in case of conflicts (1-10 scale)




CATEGORY_SCHEMA = {
    "01_DEMOGRAPHICS": {
        "keywords": ["age", "sex", "race", "ethnicity", "hispanic", "demographic", 
                     "family", "household", "relationship", "marital", "person"],
        "exclude": ["household income", "family income"],
        "priority": 5
    },
    "02_INCOME_EARNINGS": {
        "keywords": ["income", "earnings", "wage", "salary", "pay", "revenue"],
        "exclude": [],
        "priority": 7
    },
    "03_EMPLOYMENT_LABOR": {
        "keywords": ["employment", "labor", "work", "occupation", "industry", 
                     "job", "employed", "unemployed", "worker", "class of worker",
                     "hours worked", "full-time", "part-time", "indus", "occ",
                     "agriculture", "non-agriculture"],
        "exclude": [],
        "priority": 6
    },
    "04_HOUSING": {
        "keywords": ["housing", "house", "rent", "mortgage", "property", 
                     "dwelling", "residence", "apartment", "owner", "renter",
                     "real estate", "home"],
        "exclude": ["household"],
        "priority": 7
    },
    "05_EDUCATION": {
        "keywords": ["education", "school", "enrollment", "degree", "college", 
                     "university", "grade", "attainment", "diploma", "student"],
        "exclude": [],
        "priority": 7
    },
    "06_POVERTY_ASSISTANCE": {
        "keywords": ["poverty", "poor", "assistance", "welfare", "tanf", 
                     "snap", "food stamp", "medicaid", "ssi", "general assist",
                     "public assistance", "government program"],
        "exclude": [],
        "priority": 8
    },
    "07_HEALTH_DISABILITY": {
        "keywords": ["health", "insurance", "disability", "medical", "medicare", 
                     "coverage", "doctor", "dentist", "hospital", "disabled"],
        "exclude": [],
        "priority": 7
    },
    "08_ASSETS_WEALTH": {
        "keywords": ["asset", "saving", "stock", "investment", "account", 
                     "fund", "bond", "debt", "owe", "equity", "wealth",
                     "bank", "financial", "dividend", "interest", "margin"],
        "exclude": [],
        "priority": 6
    },
    "09_VETERAN_MILITARY": {
        "keywords": ["veteran", "military", "service", "armed forces", 
                     "va", "retirement pay", "army", "navy"],
        "exclude": [],
        "priority": 7
    },
    "10_GEOGRAPHY": {
        "keywords": ["geography", "fips", "state", "county", "tract", 
                     "location", "msa", "cbsa", "region", "place", "cmsa",
                     "metropolitan", "central city"],
        "exclude": ["allocation"],
        "priority": 6
    },
    "11_WEIGHTS_TECHNICAL": {
        "keywords": ["weight", "flag", "allocation", "imputation", "recode", 
                     "check item", "interviewer", "topcode", "edit", "final weight"],
        "exclude": [],
        "priority": 4
    }
}

# ==============================================================================
# CATEGORIZATION FUNCTION
# ==============================================================================

def categorize_variable(label, variable_code):
    """
    Assign a category to a Census variable based on its label.
    
    Parameters:
    -----------
    label : str
        The human-readable description of the variable
    variable_code : str
        The variable code (e.g., 'B19013_001E')
    
    Returns:
    --------
    tuple : (category, confidence_score, matched_keywords)
        - category: assigned category (e.g., '01_DEMOGRAPHICS')
        - confidence_score: number of keywords matched
        - matched_keywords: comma-separated list of matched terms
    """
    
    # Handle missing labels
    if pd.isna(label):
        return "12_OTHER", 0.0, "No label"
    
    label_lower = str(label).lower()
    category_matches = {}
    
    # Evaluate each category
    for category, info in CATEGORY_SCHEMA.items():
        score = 0
        matched_keywords = []
        
        # Check if any exclude terms disqualify this category
        excluded = False
        for exclude_word in info["exclude"]:
            if exclude_word in label_lower:
                excluded = True
                break
        
        if excluded:
            continue
        
        # Count matching keywords
        for keyword in info["keywords"]:
            if keyword in label_lower:
                score += 1
                matched_keywords.append(keyword)
        
        # If keywords matched, calculate weighted score
        if score > 0:
            weighted_score = score * info["priority"]
            category_matches[category] = {
                "score": score,
                "weighted_score": weighted_score,
                "keywords": matched_keywords
            }
    
    # Return category with highest weighted score
    if category_matches:
        best_category = max(category_matches, 
                          key=lambda x: category_matches[x]["weighted_score"])
        best_info = category_matches[best_category]
        return best_category, best_info["score"], ", ".join(best_info["keywords"])
    else:
        return "12_OTHER", 0.0, "No matches"


def main():
    """
    Main function to categorize Census variables and save results.
    """
    
    print("="*70)
    print("Census Codebook Categorization")
    print("="*70)
    print(f"Input file: {INPUT_FILE}")
    print(f"Output file: {OUTPUT_FILE}")
    print(f"Processing in chunks of {CHUNK_SIZE:,} rows")
    print("="*70)
    
    start_time = time.time()
    first_chunk = True
    total_processed = 0
    
    # Process file in chunks
    for i, chunk in enumerate(pd.read_csv(INPUT_FILE, chunksize=CHUNK_SIZE)):
        chunk_start = time.time()
        
        # Apply categorization to each row
        results = chunk.apply(
            lambda row: categorize_variable(row['label'], row['variable']), 
            axis=1
        )
        
        # Add new columns
        chunk['category'] = results.apply(lambda x: x[0])
        chunk['category_confidence'] = results.apply(lambda x: x[1])
        chunk['matched_keywords'] = results.apply(lambda x: x[2])
        
        # Save to output file
        if first_chunk:
            chunk.to_csv(OUTPUT_FILE, index=False, mode='w')
            first_chunk = False
        else:
            chunk.to_csv(OUTPUT_FILE, index=False, mode='a', header=False)
        
        # Progress update
        chunk_time = time.time() - chunk_start
        total_processed += len(chunk)
        elapsed = time.time() - start_time
        
        print(f"Chunk {i+1}: Processed {len(chunk):,} rows in {chunk_time:.1f}s")
        print(f"   Total: {total_processed:,} rows | Elapsed: {elapsed:.1f}s")
    
    total_time = time.time() - start_time
    
    print("\n" + "="*70)
    print("Processing Complete")
    print("="*70)
    print(f"Total rows processed: {total_processed:,}")
    print(f"Total time: {total_time:.1f} seconds ({total_time/60:.1f} minutes)")
    print(f"Output saved to: {OUTPUT_FILE}")
    
    # Load final results for summary statistics
    print("\n" + "="*70)
    print("Category Distribution")
    print("="*70)
    
    df_final = pd.read_csv(OUTPUT_FILE)
    
    category_counts = df_final['category'].value_counts().sort_index()
    
    print(f"\nTotal variables: {len(df_final):,}\n")
    
    for category, count in category_counts.items():
        percentage = (count / len(df_final)) * 100
        print(f"{category:<25} {count:>8,} ({percentage:>5.1f}%)")
    
    # Show confidence statistics
    print("\n" + "="*70)
    print("Categorization Confidence")
    print("="*70)
    
    high_confidence = len(df_final[df_final['category_confidence'] >= 2])
    low_confidence = len(df_final[df_final['category_confidence'] == 1])
    no_confidence = len(df_final[df_final['category_confidence'] == 0])
    
    print(f"\nHigh confidence (2+ keywords): {high_confidence:,} ({high_confidence/len(df_final)*100:.1f}%)")
    print(f"Low confidence (1 keyword):    {low_confidence:,} ({low_confidence/len(df_final)*100:.1f}%)")
    print(f"No match (OTHER category):     {no_confidence:,} ({no_confidence/len(df_final)*100:.1f}%)")
    
    # Sample variables from each category
    print("\n" + "="*70)
    print("Sample Variables by Category")
    print("="*70)
    
    for category in sorted(df_final['category'].unique()):
        sample = df_final[df_final['category'] == category].head(3)
        if len(sample) > 0:
            print(f"\n{category}:")
            for idx, row in sample.iterrows():
                print(f"  - {row['variable']}: {row['label'][:60]}")
    
    print("\n" + "="*70)
    print("Categorization complete!")
    print("="*70)

# ==============================================================================
# RUN SCRIPT
# ==============================================================================

if __name__ == "__main__":
    main()

In [ ]:
# - bayesian non-parametrics
# - MMR 
# - Make embeddings and then have clusters, make the category search transferable
# - GeoTable + LLM Query PoC